In [1]:
#==============================================================================================================================
#==============================================================================================================================
                                                #Author: Vatsal Gaurang Shah
                                                #Description: Pre-processing data for Buses
#==============================================================================================================================
#==============================================================================================================================
import pandas as pd

In [2]:
sheet_name = '376  Rout name, Stage & LL'
file_path = '/Users/vatsalshah/Documents/Workspace/SEM2/DV Project/BRT  Non BRT Route Details  Bus Stop LatLong (1).xls'

data = pd.read_excel(file_path, sheet_name=sheet_name)
filtered_columns = [col for col in data.columns if not col.startswith('Unnamed:')]
data = data[filtered_columns]

data['LAT'] = pd.to_numeric(data['LAT'], errors='coerce')
data['LONG'] = pd.to_numeric(data['LONG'], errors='coerce')
data['Stop Seq'] = pd.to_numeric(data['Stop Seq'], errors='coerce')
df=data.copy()

In [3]:
df.shape

(34089, 9)

In [4]:
df = df.dropna(subset=['LAT', 'LONG'], how='all')

In [5]:
df.shape

(34084, 9)

In [6]:
median_latitude = df['LAT'].median()
median_longitude = df['LONG'].median()
print(median_latitude,median_longitude)

18.5250397 73.8452698


In [7]:
def determine_zone(lat, lon, median_lat, median_lon):
    if lat < median_lat:
        if lon < median_lon:
            return 'Zone 1'
        else:
            return 'Zone 2'
    else:
        if lon < median_lon:
            return 'Zone 4'
        else:
            return 'Zone 3'
df['Zone'] = df.apply(lambda row: determine_zone(row['LAT'], row['LONG'], median_latitude, median_longitude), axis=1)

In [8]:
df['Capacity'] = 50
df['Speed range'] = 51
df['Direction'] = df['Route'].astype(str).apply(
    lambda x: 'Down' if x.endswith('D') else 
              'Up' if x.endswith('U') else 
              'Right' if x.endswith('R') else 'Unknown'
)
df['Coordinates'] = list(zip(df['LAT'], df['LONG']))
df.sort_values(by=['Route', 'Stop Seq'], inplace=True)

buses_data = []

for route, group in df.groupby('Route'):
    BusStopArray = group['Stop Code'].tolist()
    Source_stop = group.loc[group['Stop Seq'].idxmin(), 'Stop Code']
    Destination_Stop = group.loc[group['Stop Seq'].idxmax(), 'Stop Code']
    Direction = group['Direction'].iloc[0]
    Coordinates = group['Coordinates'].tolist()  # This should maintain the order

    buses_data.append({
        'BusID': len(buses_data) + 1,
        'RouteAccess': route,
        'BusStopArray': BusStopArray,
        'Capacity': 50,
        'Speed range': 51,
        'Direction': Direction,
        'Source stop': Source_stop,
        'Destination Stop': Destination_Stop,
        'Coordinates': Coordinates
    })

buses = pd.DataFrame(buses_data)

In [9]:
buses[buses['Direction']=='Unknown']

,BusID,RouteAccess,BusStopArray,Capacity,Speed range,Direction,Source stop,Destination Stop,Coordinates


In [10]:
buses['Zone']=df['Zone']

In [11]:
buses

,BusID,RouteAccess,BusStopArray,Capacity,Speed range,Direction,Source stop,Destination Stop,Coordinates,Zone
0,1,10-D,"[10-D-00, 10-D-01, 10-D-02, 10-D-03, 10-D-04, ...",50,51,Down,10-D-00,10-D-29,"[(18.5348383, 73.944175), (18.5335083, 73.9408...",Zone 4
1,2,10-U,"[10-U-00, 10-U-01, 10-U-02, 10-U-03, 10-U-04, ...",50,51,Up,10-U-00,10-U-26,"[(18.4993533, 73.8581423), (18.5005347, 73.859...",Zone 4
2,3,100-D,"[100-D-01, 100-D-02, 100-D-03, 100-D-04, 100-D...",50,51,Down,100-D-01,100-D-38,"[(18.576553, 73.688178), (18.5810162, 73.68770...",Zone 4
3,4,100-U,"[100-U-01, 100-U-02, 100-U-03, 100-U-04, 100-U...",50,51,Up,100-U-01,100-U-38,"[(18.52357, 73.85488), (18.5263090000003, 73.8...",Zone 4
4,5,101-D,"[101-D-00, 101-D-01, 101-D-02, 101-D-03, 101-D...",50,51,Down,101-D-00,101-D-43,"[(18.45419, 73.89066), (18.4573536, 73.8908234...",Zone 4
...,...,...,...,...,...,...,...,...,...,...
956,957,VJR4-U,"[VJR4-U-01, VJR4-U-02, VJR4-U-03, VJR4-U-04, V...",50,51,Up,VJR4-U-01,VJR4-U-33,"[(18.507995, 73.7956883), (18.507865, 73.79711...",Zone 1
957,958,VJR5-D,"[VJR5-D-01, VJR5-D-02, VJR5-D-03, VJR5-D-04, V...",50,51,Down,VJR5-D-01,VJR5-D-33,"[(18.6606594, 73.7319836), (18.6535249999998, ...",Zone 1
958,959,VJR5-U,"[VJR5-U-01, VJR5-U-02, VJR5-U-03, VJR5-U-04, V...",50,51,Up,VJR5-U-01,VJR5-U-33,"[(18.52357, 73.85488), (18.5263090000003, 73.8...",Zone 1
959,960,VJR6-D,"[VJR6-D-01, VJR6-D-02, VJR6-D-03, VJR6-D-04, V...",50,51,Down,VJR6-D-01,VJR6-D-38,"[(18.576553, 73.688178), (18.5810162, 73.68770...",Zone 1


In [12]:
import pandas as pd
updated_buses = buses.copy()


In [13]:
updated_buses.to_csv('Buses_Data_Cleaned.csv')

In [14]:
import pandas as pd

df = updated_buses.rename(columns={
    'BusID': 'BusId',
    'BusStopArray': 'BusStops',
    'Speed range': 'Speed',
    'Source stop': 'Source',
    'Destination Stop': 'Destination',    
})



In [15]:
df.drop(['Coordinates','Zone'], axis=1, inplace=True)

In [16]:
df

,BusId,RouteAccess,BusStops,Capacity,Speed,Direction,Source,Destination
0,1,10-D,"[10-D-00, 10-D-01, 10-D-02, 10-D-03, 10-D-04, ...",50,51,Down,10-D-00,10-D-29
1,2,10-U,"[10-U-00, 10-U-01, 10-U-02, 10-U-03, 10-U-04, ...",50,51,Up,10-U-00,10-U-26
2,3,100-D,"[100-D-01, 100-D-02, 100-D-03, 100-D-04, 100-D...",50,51,Down,100-D-01,100-D-38
3,4,100-U,"[100-U-01, 100-U-02, 100-U-03, 100-U-04, 100-U...",50,51,Up,100-U-01,100-U-38
4,5,101-D,"[101-D-00, 101-D-01, 101-D-02, 101-D-03, 101-D...",50,51,Down,101-D-00,101-D-43
...,...,...,...,...,...,...,...,...
956,957,VJR4-U,"[VJR4-U-01, VJR4-U-02, VJR4-U-03, VJR4-U-04, V...",50,51,Up,VJR4-U-01,VJR4-U-33
957,958,VJR5-D,"[VJR5-D-01, VJR5-D-02, VJR5-D-03, VJR5-D-04, V...",50,51,Down,VJR5-D-01,VJR5-D-33
958,959,VJR5-U,"[VJR5-U-01, VJR5-U-02, VJR5-U-03, VJR5-U-04, V...",50,51,Up,VJR5-U-01,VJR5-U-33
959,960,VJR6-D,"[VJR6-D-01, VJR6-D-02, VJR6-D-03, VJR6-D-04, V...",50,51,Down,VJR6-D-01,VJR6-D-38


In [17]:
df.to_csv('Buses.csv')